In [1]:
import math
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

In [2]:
a=pd.read_csv('silver.csv')
a['date']=(pd.to_datetime(a['date'], format='%Y-%m-%d')-pd.to_datetime('1973-01-01')).dt.days
X = a.iloc[:,[0,1,2,6]].values
Y = a.iloc[:,[3]].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [3]:
def scale_datasets(X_train, X_test):
  standard_scaler = StandardScaler()
  X_train_scaled = pd.DataFrame(standard_scaler.fit_transform(X_train))
  X_test_scaled = pd.DataFrame(standard_scaler.transform(X_test))
  return X_train_scaled, X_test_scaled
X_train_scaled, X_test_scaled = scale_datasets(X_train, X_test)

In [4]:
hidden_units1 = 160
hidden_units2 = 480
hidden_units3 = 256
learning_rate = 0.01
def build_model_using_sequential():
  model = Sequential([Dense(hidden_units1, kernel_initializer='normal', activation='relu'),Dropout(0.2),
    Dense(hidden_units2, kernel_initializer='normal', activation='relu'),Dropout(0.2),
    Dense(hidden_units3, kernel_initializer='normal', activation='relu'),
    Dense(1, kernel_initializer='normal', activation='linear')])
  return model
model = build_model_using_sequential()

In [5]:
msle = MeanSquaredLogarithmicError()
model.compile(loss=msle, optimizer=Adam(learning_rate=learning_rate), metrics=[msle])
history = model.fit(X_train_scaled, Y_train, epochs=10, batch_size=64,validation_split=0.2)

Epoch 1/10
38558/38558 ━━━━━━━━━━━━━━━━━━━━ 93s 2ms/step - loss: 0.0827 - mean_squared_logarithmic_error: 0.0827 - val_loss: 0.0702 - val_mean_squared_logarithmic_error: 0.0702
Epoch 2/10
38558/38558 ━━━━━━━━━━━━━━━━━━━━ 102s 3ms/step - loss: 0.0732 - mean_squared_logarithmic_error: 0.0732 - val_loss: 0.0714 - val_mean_squared_logarithmic_error: 0.0714
Epoch 3/10
38558/38558 ━━━━━━━━━━━━━━━━━━━━ 106s 3ms/step - loss: 0.0723 - mean_squared_logarithmic_error: 0.0723 - val_loss: 0.0729 - val_mean_squared_logarithmic_error: 0.0729
Epoch 4/10
38558/38558 ━━━━━━━━━━━━━━━━━━━━ 104s 3ms/step - loss: 0.0715 - mean_squared_logarithmic_error: 0.0715 - val_loss: 0.0654 - val_mean_squared_logarithmic_error: 0.0654
Epoch 5/10
38558/38558 ━━━━━━━━━━━━━━━━━━━━ 112s 3ms/step - loss: 0.0710 - mean_squared_logarithmic_error: 0.0710 - val_loss: 0.0649 - val_mean_squared_logarithmic_error: 0.0649
Epoch 6/10
38558/38558 ━━━━━━━━━━━━━━━━━━━━ 103s 3ms/step - loss: 0.0705 - mean_squared_logarithmic_error: 0.07

In [7]:
y_1 = model.predict(X_test)
print(y_1)  
y_1 = pd.DataFrame(y_1)
y_1.to_csv('RF-EQPD.csv')

24099/24099 ━━━━━━━━━━━━━━━━━━━━ 35s 1ms/step
[[1.0402846e+00]
 [7.6613745e+03]
 [1.0850110e+04]
 ...
 [1.5813071e+03]
 [1.2171315e+04]
 [9.9614951e+03]]


In [7]:
print("MSE: ",mean_squared_error(Y_test,y_1)," \nR2: ",r2_score(Y_test,y_1)," \nMAE: ",mean_absolute_error(Y_test,y_1))

MSE:  26.02709818867845  
R2:  -14.20033442064869  
MAE:  1.3914766810030201
